# AdaBoost

当做重要决定时，大家可能都会考虑吸取多个专家而不只是一个人的意见。机器学习处理问题时又何尝不是如此？这就是元算法（meta-algorithm）背后的思路。元算法是对其他算法进行组 合的一种方式。接下来我们将集中关注一个称作AdaBoost的最流行的元算法。由于某些人认为 AdaBoost是最好的监督学习的方法，所以该方法是机器学习工具箱中最强有力的工具之一。

非均衡分类问题。当我们试图对样例数目不均衡的数据进行分类时，就会遇到这个问题。信用卡使用中的欺 诈检测就是非均衡问题中的一个极好的例子，此时我们可能会对每一个正例样本都有1000个反例样本。在这种情况下，分类器将如何工作？读者将会了解到，可能需要利用修改后的指标来评价分类器的性能。

## 基于数据集多重抽样的分类器

我们自然可以将不同的分类器组合起来，而这种组合结果则被称为集成方法（ensemble method）或者元算法（meta-algorithm）。使 用集成方法时会有多种形式，
- 不同算法的集成，
- 同一算法在不同设置下的集成，
- 数据集不同部分分配给不同分类器之后的集成。

我们将介绍基于同一种分类器 多个不同实例的两种计算方法。在这些方法当中，数据集也会不断变化，而后应用于不同的实例 分类器上。

adaboost

优点：泛化错误率低，易编码，可以应用在大部分分类器上，无参数调整。

缺点：对离群点敏感。

适用数据类型：数值型和标称型数据。

### bagging：基于数据随机重抽样的分类器构建方法

自举汇聚法（bootstrap aggregating），也称为bagging方法，是在从原始数据集选择S次后 得到S个新数据集的一种技术。新数据集和原数据集的大小相等。bagging 中的数据集通常被认为是放回取样得到的，比如要得到一个大小为n的新数据集，该数据集中的每个样本都是在 原始数据集中随机抽样（即抽样之后又放回）得到的。这里的替换就意味着可以多次地选择同一 样本。 这一性质就允许新数据集中可以有重复的值， 而原始数据集的某些值在新集合中则不再出现。

在S个数据集建好之后，将某个学习算法分别作用于每个数据集就得到了S个分类器。当我们要对新数据进行分类时，就可以应用这S个分类器进行分类。与此同时，选择分类器投票结果中最多的类别作为最后的分类结果。

当然，还有一些更先进的bagging方法，比如随机森林（random forest）。有关这些方法的一 个很好的讨论材料参见网页http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm。接 下来我们将注意力转向一个与bagging类似的集成分类器方法boosting。

### boosting

boosting是一种与bagging很类似的技术。不论是在boosting还是bagging当中，所使用的多个分类器的类型都是一致的。但是在boosting中，不同的分类器是通过串行训练而获得的，每个新分类器都根据已训练出的分类器的性能来进行训练。boosting是通过集中**关注被已有分类器错分的那些数据来获得新的分类器**。

由于boosting分类的结果是基于**所有分类器的加权求和结果的**，因此boosting与bagging不太一 样。**bagging中的分类器权重是相等的，而boosting中的分类器权重并不相等**，每个权重代表的是其对应分类器在上一轮迭代中的成功度。boosting方法拥有多个版本，本章将只关注其中一个最流行的版本AdaBoost。

### AdaBoost的一些理论

能否使用弱分类器和多个实例来构建一个强分类器？这是一个非常有趣的理论问题。这里的 “弱”意味着分类器的性能比随机猜测要略好，但是也不会好太多。这就是说，在二分类情况下 弱分类器的错误率会高于50%，而“强”分类器的错误率将会低很多。AdaBoost算法即脱胎于上 述理论问题。

AdaBoost是adaptive boosting（自适应boosting）的缩写，其运行过程如下：训练数据中的每 个样本，并赋予其一个权重，这些权重构成了向量D。一开始，这些权重都初始化成相等值。首先在训练数据上训练出一个弱分类器并计算该分类器的错误率，然后在**同一数据集上再次训练弱分类器**。在分类器的第二次训练当中，**将会重新调整每个样本的权重**，其中第一次分对的样本的权重将会降低，而第一次分错的样本的权重将会提高。为了从所有弱分类器中得到最终的分类结果，AdaBoost为每个分类器都分配了一个权重值alpha，这些alpha值是基于每个弱分类器的错误率进行计算的。其中，错误率ε的定义为：

$$\varepsilon=\frac{未正确分类的样本数量}{所有的样本数量} $$

$$\alpha=\frac{1}{2} \ln \left(\frac{1-\varepsilon}{\varepsilon}\right)$$

在加权训练样本上面，误差率定义为被错误分类样本的权值之和，也就是代码中。 错误向量errArr和权重向量D的相应元素相乘并求和，就得到了数值weightedError

<img src="EXTRAS/pic.png" style="width:400;height:400px;">

AdaBoost算法的示意图。左边是数据集，其中直方图的不同宽度表示每个样例上的不同权重。 在经过一个分类器之后， 加权的预测结果会通过三角形中的 alpha值进行加权。每个三角形中输出的加权结果在圆形中求和，从而得到最终 的输出结果。

计算出alpha值之后，可以对权重向量D（每个样本的权重）进行更新，以使得那些正确分类的样本的权重降低而 错分样本的权重升高。D的计算方法如下

如果一个样本分类正确：$$D_{i}^{(t+1)}=\frac{D_{i}^{(t)} \mathrm{e}^{-\alpha}}{\operatorname{Sum}(D)}$$

如果一个样本分类错误：$$D_{i}^{(t+1)}=\frac{D_{i}^{(t)} \mathrm{e}^{\alpha}}{\operatorname{Sum}(D)}$$

在计算出D之后，AdaBoost又开始进入下一轮迭代。AdaBoost算法会不断地重复训练和调整 权重的过程，直到训练错误率为0或者弱分类器的数目达到用户的指定值为止。

这里使用了基于单层决策树构建弱分类器，单层决策树（decision stump，也称决策树桩）是一种简单的决策树。前面我们已经介绍了决策树的工作原理，接下来将构建一个单层决策树，而它仅基于单个特征来做决策。由于这棵树只 有一次分裂过程，因此它实际上就是一个树桩。

单层决策树生成函数伪代码

    将最小错误率minError设为+∞ 
    对数据集中的每一个特征（第一层循环）：
        对每个步长（第二层循环）：
            对每个不等号（第三层循环）： 
                建立一棵单层决策树并利用加权数据集对它进行测试 
                如果错误率低于minError，则将当前单层决策树设为最佳单层决策树 
    返回最佳单层决策树

In [1]:
import adaboost
dateingMat, label = adaboost.loadSimpData()
import matplotlib.pyplot as plt
from numpy import *
fig = plt.figure()

dateingMat = array(dateingMat)
label = array(label)
ax = fig.add_subplot(111)
ax.scatter(dateingMat[:,0],dateingMat[:,1], 20.0*array(label+2), 20.0*array(label+1))
plt.show()

<Figure size 640x480 with 1 Axes>

### 这里构建树的指标是加权数据的错误率

In [2]:
D= mat(ones((5,1)) /5)
res = adaboost.buildStump(dateingMat,label, D)
res

split: dim 0, thresh 0.90, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 0.90, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.00, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.00, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.10, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.10, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.20, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.20, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.30, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.30, thresh ineqal: gt, the weighted error is 0.800
split: dim 0, thresh 1.40, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.40, thresh ineqal: gt, the weighted error is 0.800
split: dim 0, thresh 1.50, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.50, thresh ineq

({'dim': 0, 'thresh': 1.3, 'ineq': 'lt'}, matrix([[0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

## 完整 AdaBoost 算法的实现

我们构建了一个基于加权输入值进行决策的分类器。现在，我们拥有了实现一个 完整AdaBoost算法所需要的所有信息。我们将利用7.3节构建的单层决策树来实现7.2节中给出提纲的算法。

    对每次迭代：

        -利用buildStump()函数找到最佳的单层决策树 
        -将最佳单层决策树加入到单层决策树数组 
        -计算alpha 
        -计算新的权重向量D 
        -更新累计类别估计值 
        -如果错误率等于0.0，则退出循环

In [3]:
classfierArray = adaboost.adaBoostTrainDS(dateingMat, label, 9)

split: dim 0, thresh 0.90, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 0.90, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.00, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.00, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.10, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.10, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.20, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.20, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.30, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.30, thresh ineqal: gt, the weighted error is 0.800
split: dim 0, thresh 1.40, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.40, thresh ineqal: gt, the weighted error is 0.800
split: dim 0, thresh 1.50, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.50, thresh ineq

AdaBoost算法的输入参数包括数据集、类别标签以及迭代次数numIt，其中numIt是在整个 AdaBoost算法中唯一需要用户指定的参数。

我们假定迭代次数设为9，如果算法在第三次迭代之后错误率为0，那么就会退出迭代过程， 因此，此时就不需要执行所有的9次迭代过程。

函数名称尾部的DS代表的就是单层决策树（decision stump），它是AdaBoost中最流行的弱分类器，当然并非唯一可用的弱分类器。上述函数确实是建立于单层决策树之上的，但是我们也可 以很容易对此进行修改以引入其他基分类器。实际上，任意分类器都可以作为基分类器，本书前 面讲到的任何一个算法都行。上述算法会输出一个单层决策树的数组，因此首先需要建立一个新 的Python表来对其进行存储。然后，得到数据集中的数据点的数目m，并建立一个列向量D。

向量D非常重要，它包含了每个数据点的权重。一开始，这些权重都赋予了相等的值。在后续的迭代中，AdaBoost算法会在增加错分数据的权重的同时，降低正确分类数据的权重。D是一个概率分布向量，因此其所有的元素之和为1.0。为了满足此要求，一开始的所有元素都会被初始化成1/m。同时，程序还会建立另一个列向量aggClassEst，记录每个数据点的类别估计累计值(累计所有决策树结果的预测误差)。

AdaBoost算法的核心在于for循环，该循环运行numIt次或者直到训练错误率为0为止。循环中的第一件事就是利用前面介绍的buildStump()函数建立一个单层决策树。该函数的输入为权 重向量D，返回的则是利用D而得到的具有最小错误率的单层决策树，同时返回的还有最小的错误率以及估计的类别向量。

接下来，需要计算的则是alpha值。该值会告诉总分类器本次单层决策树输出结果的权重。其 中的语句max(error, 1e-16)用于确保在没有错误时不会发生除零溢出。而后，alpha值加入到 bestStump字典中，该字典又添加到列表中。该字典包括了分类所需要的所有信息。

接下来的三行 则用于计算下一次迭代中的新权重向量D。在训练错误率为0时，就要提前 结束for循环。此时程序是通过aggClassEst变量保持一个运行时的类别估计值来实现的 。该 值只是一个浮点数，为了得到二值分类结果还需要调用sign()函数。如果总错误率为0，则由 break语句中止for循环。

### 预测算法：基于 AdaBoost 的分类

需要做的就只是将弱分 类器的训练过程从程序中抽出来，然后应用到某个具体的实例上去。每个弱分类器的结果以其对 应的alpha值作为权重。所有这些弱分类器的结果加权求和就得到了最后的结果。大于0则返回+1，而如果小于0则返回1

In [4]:
classfierArray = adaboost.adaBoostTrainDS(dateingMat, label, 30)
adaboost.adaClassify ([0,0] ,classfierArray) 
#可以发现，随着迭代的进行，数据点[0,0]的分类结果越来越强。之前是迭代9次

split: dim 0, thresh 0.90, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 0.90, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.00, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.00, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.10, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.10, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.20, thresh ineqal: lt, the weighted error is 0.400
split: dim 0, thresh 1.20, thresh ineqal: gt, the weighted error is 0.600
split: dim 0, thresh 1.30, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.30, thresh ineqal: gt, the weighted error is 0.800
split: dim 0, thresh 1.40, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.40, thresh ineqal: gt, the weighted error is 0.800
split: dim 0, thresh 1.50, thresh ineqal: lt, the weighted error is 0.200
split: dim 0, thresh 1.50, thresh ineq

matrix([[-1.]])

### 马疝病数据集上应用AdaBoost分类器

使用adaboost

(1) 收集数据：提供的文本文件。

(2) 准备数据：确保类别标签是+1和1而非1和0。

(3) 分析数据：手工检查数据。

(4) 训练算法：在数据上，利用adaBoostTrainDS()函数训练出一系列的分类器。

(5) 测试算法：我们拥有两个数据集。在不采用随机抽样的方法下，我们就会对AdaBoost和Logistic回归的结果进行完全对等的比较。

(6) 使用算法：观察该例子上的错误率。不过，也可以构建一个Web网站，让驯马师输入马的症状然后预测马是否会死去。

In [5]:
import adaboost
from numpy import *
data, labelArr = adaboost.loadDataSet('horseColicTraining2.txt')
classifierArray = adaboost.adaBoostTrainDS(data,labelArr,10, isPrint=False)
testArr, testLabelArr = adaboost.loadDataSet('horseColicTest2.txt')
prediction10 = adaboost.adaClassify(testArr, classifierArray)
errArr = mat(ones((67,1)))
errArr[prediction10 != mat(testLabelArr).T].sum()
#分类错误的数量

16.0

很多人都认为，AdaBoost和SVM是监督机器学习中最强大的两种方法。实际上，这两者之间拥有不少相似之处。我们可以把弱分类器想象成SVM中的一个核函数，也可以按照最大化某个最 小间隔的方式重写AdaBoost算法。而它们的不同就在于其所定义的间隔计算方式有所不同，因此 导致的结果也不同。特别是在高维空间下，这两者之间的差异就会更加明显。

### 疑问： 每次的分类器的参数是根据自己训练的， 最后的预测结果是每个分类起的结果乘对应的权重之和，这个数会不会很大？？这里每个分类器的权重之和不是1？？？

### 非均衡分类问题

错误率指的是在所有测 试样例中错分的样例比例。实际上，这样的度量错误掩盖了样例如何被分错的事实。
**当某个类别的重要性高于其他类别时**，我们就可以利用上述定义来定义出多个比错误率更好的新指标。
正确率（Precision），它等于TP/(TP+FP)，给出的是预测为正例的真实正例占预测为正例的比例。 - 尽可能少预测错
召回率（Recall），它等于TP/(TP+FN)，给出的是预测为正例的真实正例占所有真实正例的比例。  - 情愿误判也不漏判
这俩都关心的是正例，在召回率很大的分类器中，真正判错的正例的数目并不多。

|      | +1           | -1           |
| ---- | ------------ | ------------ |
| +1   | 真正例（TP） | 伪反例（FN） |
| -1   | 伪正例（FP） | 真反例（TN） |

另一个用于度量分类中的非均衡性的工具是ROC曲线（ROC curve），ROC代表接收者操作特征（receiver operating characteristic）横轴是伪正例的比例（假阳率=FP/(FP+TN)），而纵轴是真正例的比例（真阳率=TP/(TP+FN)）。左下角的点所对应的是将所有样例判为反例的情况，而右上 角的点对应的则是将所有样例判为正例的情况。虚线给出的是随机猜测的结果曲线
<img src="EXTRAS/pic 2.png" style="width:400;height:400px;">

ROC曲线不但可以用于比较分类器，还可以基于成本效益（cost-versus-benefit）分析来做出决策。由于在不同的阈值下，不同的分类器的表现情况可能各不相同，因此以某种方式将它们组合起来或许会更有意义。如果只是简单地观察分类器的错误率，那么我们就难以得到这种更深入的洞察效果了。

在理想的情况下，最佳的分类器应该尽可能地处于左上角，这就意味着分类器在假阳率很低的同时获得了很高的真阳率。例如在垃圾邮件的过滤中，这就相当于过滤了所有的垃圾邮件，但 没有将任何合法邮件误识为垃圾邮件而放入垃圾邮件的文件夹中。

对不同的ROC曲线进行比较的一个指标是曲线下的面积（Area Unser the Curve，AUC）。AUC 给出的是分类器的平均性能值，当然它并不能完全代替对整条曲线的观察。一个完美分类器的 AUC为1.0，而随机猜测的AUC则为0.5。

代码中画roc的方式是根据补偿，首先对预测值进行排序， 根据标签， ，每得到一个标签为1.0的类，则要沿着y轴 的方向下降一个步长，即不断降低真阳率。类似地，对于每个其他类别的标签，则是在x轴方向 上倒退了一个步长（假阴率方向）。 从<1,1> 不断移动到 <0,0> 

## 基于代价函数的分类器决策控制

除了调节分类器的阈值之外，我们还有一些其他可以用于处理非均衡分类代价的方法，其中 的一种称为代价敏感的学习（cost-sensitive learning）。考虑表7-4中的代价矩阵，第一张表给出的 是到目前为止分类器的代价矩阵（代价不是0就是1）。我们可以基于该代价矩阵计算其总代价： $TP*0+FN*1+FP*1+TN*0$。接下来我们考虑下面的第二张表，基于该代价矩阵的分类代价的计算 公式为：$TP*(-5)+FN*1+FP*50+TN*0$。采用第二张表作为代价矩阵时，两种分类错误的代价是 不一样的。类似地，这两种正确分类所得到的收益也不一样。如果在构建分类器时，知道了这些代价值，那么就可以选择付出最小代价的分类器。

<img src="EXTRAS/pic 3.png" style="width:400;height:400px;">

在分类算法中，我们有很多方法可以用来引入代价信息。在AdaBoost中，可以基于代价函数 来调整错误权重向量D。在朴素贝叶斯中，可以选择具有最小期望代价而不是最大概率的类别作 为最后的结果。在SVM中，可以在代价函数中对于不同的类别选择不同的参数C。上述做法就会 给较小类更多的权重，即在训练时，小类当中只允许更少的错误。


### 处理非均衡问题的数据抽样方法
另外一种针对非均衡问题调节分类器的方法，就是对分类器的训练数据进行改造。这可以通 过欠抽样（undersampling）或者过抽样（oversampling）来实现。过抽样意味着复制样例，而欠 抽样意味着删除样例。不管采用哪种方式，数据都会从原始形式改造为新形式。抽样过程则可以 通过随机方式或者某个预定方式来实现。

通常也会存在某个罕见的类别需要我们来识别，比如在信用卡欺诈当中。如前所述，正例类别属于罕见类别。我们希望对于这种罕见类别能尽可能保留更多的信息，因此，我们应该保留正 例类别中的所有样例，而对反例类别进行欠抽样或者样例删除处理。这种方法的一个缺点就在于 要确定哪些样例需要进行剔除。但是，在选择剔除的样例中可能携带了剩余样例中并不包含的有 价值信息。

上述问题的一种解决办法，就是选择那些离决策边界较远的样例进行删除。假定我们有一个 数据集，其中有50例信用卡欺诈交易和5000例合法交易。如果我们想要对合法交易样例进行欠抽样处理，使得这两类数据比较均衡的话，那么我们就需要去掉4950个样例，而这些样例中可能包含很多有价值的信息。这看上去有些极端，因此有一种替代的策略就是使用反例类别的欠抽样和 正例类别的过抽样相混合的方法。

要对正例类别进行过抽样，我们可以复制已有样例或者加入与已有样例相似的点。一种方法 是加入已有数据点的插值点，但是这种做法可能会导致过拟合的问题。

## 小结

集成方法通过组合多个分类器的分类结果，获得了比简单的单分类器更好的分类结果。有一些利用不同分类器的集成方法，但是本章只介绍了那些利用同一类分类器的集成方法。

多个分类器组合可能会进一步凸显出单分类器的不足，比如过拟合问题。如果分类器之间差别显著，那么多个分类器组合就可能会缓解这一问题。分类器之间的差别可以是算法本身或者是应用于算法上的数据的不同。

本章介绍的两种集成方法是bagging和boosting。在bagging中，是通过随机抽样的替换方式， 得到了与原始数据集规模一样的数据集。而boosting在bagging的思路上更进了一步，它在数据集上顺序应用了多个不同的分类器。另一个成功的集成方法就是随机森林，但是由于随机森林不如AdaBoost流行，所以本书并没有对它进行介绍。

本章介绍了boosting方法中最流行的一个称为AdaBoost的算法。AdaBoost以弱学习器作为基 分类器，并且输入数据，使其通过权重向量进行加权。在第一次迭代当中，所有数据都等权重。 但是在后续的迭代当中，前次迭代中分错的数据的权重会增大。这种针对错误的调节能力正是 AdaBoost的长处。

本章以单层决策树作为弱学习器构建了AdaBoost分类器。实际上，AdaBoost函数可以应用于任意分类器，只要该分类器能够处理加权数据即可。AdaBoost算法十分强大，它能够快速处理其 他分类器很难处理的数据集。

非均衡分类问题是指在分类器训练时正例数目和反例数目不相等（相差很大）。该问题在错分正例和反例的代价不同时也存在。本章不仅考察了一种不同分类器的评价方法——ROC曲线， 还介绍了正确率和召回率这两种在类别重要性不同时，度量分类器性能的指标。

本章介绍了通过过抽样和欠抽样方法来调节数据集中的正例和反例数目。另外一种可能更好的非均衡问题的处理方法，就是在训练分类器时将错误的代价考虑在内。